In [566]:
import librosa
import numpy as np
import torch
import pandas as pd

from src.constants import SAMPLE_RATE, WINDOW_LENGTH

HOP_LENGTH = 20
HOP_LENGTH = int(HOP_LENGTH / 1000 * SAMPLE_RATE)
SAMPLE_RATE = 16000

audio, _ = librosa.load("input/j3ko.wav", sr=SAMPLE_RATE)
audio_l = len(audio)


audio = torch.from_numpy(audio).float()
audio_steps = audio_l // HOP_LENGTH + 1

seq_len = 2.55
seq_len = int(seq_len * SAMPLE_RATE)

chunk_duration = 2.55
overlap = 1.5
chunk_samples = int(chunk_duration * SAMPLE_RATE)
chunk_overlap = int(chunk_duration * SAMPLE_RATE  // overlap)
chunks = []
# print(len(audio))
# print(len(audio) - chunk_samples)
for i in range(0, len(audio) - chunk_samples + 1, chunk_overlap):
    chunk = audio[i:i + chunk_samples]
    print('end =', i + chunk_samples)
    print('padleft = ', i)
    print('padright = ', len(audio)-i-chunk_samples)
    print('chunk shape = ', chunk.shape)
    print('-------')
    chunk = torch.nn.functional.pad(chunk, (i, len(audio)-i-chunk_samples), 'constant', 0)
    chunk = np.pad(chunk, WINDOW_LENGTH // 2, mode="constant")        
    chunks.append(chunk)


datas = []

for chunk in chunks :
    data = []
    for i in range(len(chunk) // seq_len):
        begin_t = i * seq_len
        end_t = begin_t + seq_len + WINDOW_LENGTH
        data.append(
            chunk[begin_t:end_t],
        )
    chunk = chunk[end_t - WINDOW_LENGTH :]
    padding_size = seq_len + WINDOW_LENGTH - len(chunk)
    data.append(
        np.pad(chunk, (0, padding_size), mode='constant'),
    )
    datas.append(data)

datas = torch.from_numpy(np.array(datas)).float()
chunk_n = datas.shape[1]

end = 40800
padleft =  0
padright =  263673
chunk shape =  torch.Size([40800])
-------
end = 68000
padleft =  27200
padright =  236473
chunk shape =  torch.Size([40800])
-------
end = 95200
padleft =  54400
padright =  209273
chunk shape =  torch.Size([40800])
-------
end = 122400
padleft =  81600
padright =  182073
chunk shape =  torch.Size([40800])
-------
end = 149600
padleft =  108800
padright =  154873
chunk shape =  torch.Size([40800])
-------
end = 176800
padleft =  136000
padright =  127673
chunk shape =  torch.Size([40800])
-------
end = 204000
padleft =  163200
padright =  100473
chunk shape =  torch.Size([40800])
-------
end = 231200
padleft =  190400
padright =  73273
chunk shape =  torch.Size([40800])
-------
end = 258400
padleft =  217600
padright =  46073
chunk shape =  torch.Size([40800])
-------
end = 285600
padleft =  244800
padright =  18873
chunk shape =  torch.Size([40800])
-------


In [567]:
len(audio) / seq_len

7.462573529411765

In [568]:
len(chunk[end_t - WINDOW_LENGTH :])/seq_len

0.0

In [569]:
(len(audio) - end_t + WINDOW_LENGTH)/seq_len

0.4625735294117647

In [570]:
datas.shape[1]

8

In [571]:
len(chunks)

10

In [572]:
from src.model import E2E


model = torch.load('runs/Pitch_FL6_0/model.pt')
model.eval()
model = model.cuda()

In [573]:
results = []
overlap_results = []
i = 0
for data in datas:
    data = data.cuda()
    with torch.no_grad():
        result = model(data)
        results.append(result[1].cpu().numpy())
    i += 1

results = torch.from_numpy(np.array(results)).float()

In [574]:
results.shape

torch.Size([10, 8, 128, 360])

In [575]:
from src.utils import to_local_average_cents

cents_pred_chunk = []

for result in results:
    print(result.shape)
    cents_pred = to_local_average_cents(result.view(-1,360).cpu().numpy(),None, 0 )
    cents_pred_chunk.append(cents_pred)

torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])


In [576]:
cents_pred_chunk[0].shape[0] / 8

128.0

In [577]:
cents_pred_chunk[0].shape

(1024,)

In [578]:
freq_l // chunk_n

128

In [579]:
freqs = []
freq_l = cents_pred_chunk[0].shape[0]
freq_chunk_size = freq_l // chunk_n
overlap_chunk =  int(freq_chunk_size // overlap)
for i, cents_pred in enumerate(cents_pred_chunk):
    freq = np.array(
        [10 * (2 ** (cent / 1200)) if cent else 0 for cent in cents_pred]
    )
    st = max(0,i*overlap_chunk)
    print(f'{st}: {st+freq_chunk_size}')
    freq_pad = np.pad(freq[st: st+freq_chunk_size], (st, freq_l - (st+freq_chunk_size)), mode="constant")
    freqs.append(freq_pad)

freqs = np.array(freqs)
freqs[freqs == 0] = np.nan
freq_result = np.nanmean(freqs, axis=0)

0: 128
85: 213
170: 298
255: 383
340: 468
425: 553
510: 638
595: 723
680: 808
765: 893


C:\Users\putte\AppData\Local\Temp\ipykernel_23992\1199437621.py:19: RuntimeWarning: Mean of empty slice
  freq_result = np.nanmean(freqs, axis=0)


In [582]:
df = pd.DataFrame({"frequency": freq_result})

In [583]:
df.to_csv('f0.csv', index=False)

In [584]:
import pandas as pd
import numpy as np
from scipy.io.wavfile import write

f0 = pd.read_csv("f0.csv")


def nco(fcw, sr):
    phase = 0
    phase_result = []
    for fcw_samp in fcw:
        ph_step = 2 * np.pi * fcw_samp * 1 / sr
        phase += ph_step
        phase_result.append(phase)
    return np.cos(phase_result)


step_size = 20
SAMPLE_RATE = 16000
quantum = (SAMPLE_RATE * step_size) // 1000
# sample_rate = quantum * (1000 // step_size)

fcw = np.zeros(quantum * f0.shape[0])

for i in range(f0.shape[0]):
    fcw[quantum * (i) : quantum * (i + 1)] = f0["frequency"][i]

output = nco(fcw, SAMPLE_RATE)



write("j3k.wav", SAMPLE_RATE, output)